In [ ]:
#!/usr/bin/env python3 
# -*- coding: utf-8 -*-
# ======================================================================
# Created By  : Yuki Ishikawa
# Created Date: 2024/10/19
# Edited on 2025/02/25
# - Fig2 and Fig3 were re-visuazlized for revision
# ======================================================================
"""
Draw figures for the manuscript
"""
# ======================================================================
# Imports
# ======================================================================
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from scipy import stats

plt.rcParams["font.family"] = "Times New Roman"

In [ ]:
# ======================================================================
# Settings
# ======================================================================
## Experiments
exps = ["conventional", "virtual"]

## I/O
idir = "./dat"
odir = "./fig"

# ======================================================================
# Functions
# ======================================================================
def rbias_func(y_true, y_pred):
    """
    Args:
        y_true (array): Array of observed values
        y_pred (array): Array of predicted values
    Returns:
        rbias (float): relative bias score
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_diff = y_pred - y_true
    rbias = y_diff.sum() / y_true.sum()
    return rbias

def nse_func(y_true, y_pred):
    """
    Args:
        y_true (array): Array of observed values
        y_pred (array): Array of predicted values
    Returns:
        nse (float): Nash-sutcliffe efficiency
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_numerator = (y_true - y_pred) ** 2
    y_denominator = (y_true - y_true.mean()) ** 2
    nse = 1 - y_numerator.sum() / y_denominator.sum()
    return nse

In [ ]:
# ======================================================================
# Figure 2
# ======================================================================
for exp in exps:
    ## Load the results
    df = pd.read_csv(f"{idir}/reach_results_{exp}.csv")
    df = df[df['SectionID'] != 0].reset_index(drop=True)
    ## Settings for visualization
    xticks = np.arange(0, int(max(df['RivmouDist'])), 500000)
    ## Draw the figure
    fig = plt.figure(figsize=(15,10), facecolor='w', dpi=100, tight_layout=True)
    ## Upper panel (Spatial discharge changes)
    ax1 = fig.add_axes((0.05,0.30,0.9,0.675))
    ax1.invert_xaxis()
    ## BAM predicted discharge
    for sec_id in df['SectionID'].unique():
        df_sec = df[df['SectionID'] == sec_id]
        if sec_id == 1:
            ax1.plot(df_sec['RivmouDist'], df_sec['DisPreMean'], linewidth=1, color='skyblue', label=f'BAM org.', zorder=2)
            ax1.plot(df_sec['RivmouDist'], df_sec['DisPreReg'], linewidth=2, linestyle='dashdot', color='b', label=f'BAM reg.', zorder=4)
        else:
            ax1.plot(df_sec['RivmouDist'], df_sec['DisPreMean'], linewidth=1, color='skyblue', zorder=2)
            ax1.plot(df_sec['RivmouDist'], df_sec['DisPreReg'], linewidth=2, linestyle='dashdot', color='b', zorder=4)
    ## Prior discharge (GRADES)
    ax1.plot(df['RivmouDist'], df['DisAprMean'], linewidth=2, linestyle='--', color='dimgray', label=f'Prior', zorder=4)
    ## Observed discharge
    df_obs = df[df['FlagObs'] == 1]
    ax1.scatter(df_obs['RivmouDist'], df_obs['DisObsMean'], marker="x", s=120, c='black', label='Gauge', zorder=5)
    ## Plot reservoir info
    df_res = df[df['FlagRes'] == 1]
    df_res_pre = df.iloc[df_res.index - 1, :]
    texts = ['D1', 'C1,D2', 'D3', 'D4', 'D5']
    text_loc_x = np.array([0, 1/11, 3/11, 4/11, 9/11]) + 0.05
    arrow_loc_x = np.array([0.282, 0.337, 0.44, 0.496, 0.769])
    j = 0
    for i, row in df_res.iterrows():
        if j == 1: # D2 
            bottom = 0
            top = bottom + 100
            while top <= 2500:
                ax1.plot(np.full(2, row['RivmouDist']), [bottom,top], linewidth=3.5, color='orange', alpha=0.3, zorder=1)
                bottom = top
                top = bottom + 100
                ax1.plot(np.full(2, row['RivmouDist']), [bottom,top], linewidth=3.5, color='dimgray', alpha=0.3, zorder=1)
                bottom = top
                top = bottom + 100
        else:
            ax1.plot(np.full(2, row['RivmouDist']), [0,2500], linewidth=3.5, color='orange', alpha=0.3, zorder=1)
        ax1.annotate(text=texts[j], xy=(text_loc_x[j], 1.13), xycoords='axes fraction', size=30, ha='center')
        # ax1.annotate(text=f"{row['DisPreMean'] - df_res_pre.loc[i-1, 'DisPreMean']:.1f}", xy=(arrow_loc_x[j], 1.005), xytext=(text_loc_x[j], 1.07), xycoords='axes fraction', size=28, ha='center')
        ax1.annotate(text=None, xy=(arrow_loc_x[j], 1.005), xytext=(text_loc_x[j], 1.06), xycoords='axes fraction', arrowprops=dict(headlength=0.3, headwidth=0.2, width=0.05, color='darkgray'), size=18, ha='center')
        j += 1
    ## Plot confluence info
    df_con = df[df['FlagConf'] == 1].iloc[1:,:]
    df_con_pre = df.iloc[df_con.index - 1, :]
    text_loc_x = np.array([1/11, 2/11, 5/11, 6/11, 7/11, 8/11, 10/11]) + 0.05
    arrow_loc_x = np.array([0.337, 0.345, 0.532, 0.680, 0.724, 0.741, 0.804])
    j = 0
    for i, row in df_con.iterrows():
        if j != 0:
            ax1.plot(np.full(2, row['RivmouDist']), [0,2500], linewidth=3.5, color='dimgray', alpha=0.3, zorder=1)
            ax1.annotate(text=f'C{j+1}', xy=(text_loc_x[j], 1.13), xycoords='axes fraction', size=30, ha='center')
            ax1.annotate(text=f"{row['DisPreMean'] - df_con_pre.loc[i-1, 'DisPreMean']:.1f}", xy=(arrow_loc_x[j], 1.005), xytext=(text_loc_x[j], 1.07), xycoords='axes fraction', size=28, ha='center')
            ax1.annotate(text= None, xy=(arrow_loc_x[j], 1.005), xytext=(text_loc_x[j], 1.06), xycoords='axes fraction', arrowprops=dict(headlength=0.3, headwidth=0.2, width=0.05, color='darkgray'), size=18, ha='center')
        j += 1
    ## Plot settings 
    ax1.legend(loc='upper left', fontsize=28)
    ax1.set_xticks(xticks)
    ax1.set_xticklabels(xticks/1000000)
    ax1.set_ylim([0,2500])
    ax1.set_ylabel('Discharge [$m^3/s$]', size=40, math_fontfamily='cm')
    ax1.tick_params(labelsize=40)

    ## Lower panel (HID irrigated area)
    ax2 = fig.add_axes((0.05,0.05,0.9,0.240))
    ax2.invert_xaxis()
    for sec_id in df['SectionID'].unique():
        df_sec = df[df['SectionID'] == sec_id]
        x = [df_sec['RivmouDist'].iloc[0], df_sec['RivmouDist'].iloc[0], df_sec['RivmouDist'].iloc[-1], df_sec['RivmouDist'].iloc[-1]]
        y = [0, df_sec['AreaHID'].sum()/100, df_sec['AreaHID'].sum()/100, 0]
        ax2.fill(x, y, color='palegreen')
    ax2.set_xticks(xticks)
    ax2.set_xticklabels(xticks/1000000)
    ax2.set_yticks([0, 200, 400, 600])
    ax2.set_yticklabels([0, 200, 400, 600])
    ax2.set_xlabel('Distance from river mouth [$10^3 km$]', size=40, math_fontfamily='cm')
    ax2.set_ylabel('Irrg. area per\nsection [$km^2$]', size=40, math_fontfamily='cm')
    ax2.tick_params(labelsize=40)
    ax2.grid(axis='y', linestyle='dotted', zorder=1)

    ## Save figure
    plt.savefig(f"{odir}/Fig3_{exp}.png")

In [ ]:
# ======================================================================
# Figure 3a: Hydrographs at selected gauges
# ======================================================================
## Empty dataframe to save the evaluation result
df = pd.DataFrame(index=["CC_conventional", "rBIAS_conventional", "NSE_conventional", "CC_virtual", "rBIAS_virtual", "NSE_virtual"])
## Draw the figure
fig = plt.figure(figsize=(18,12), facecolor='w', dpi=100, tight_layout=True)
i_vis = 0
for i in range(19):
    ## Load the observed discharge
    q_obs = pd.read_csv(f"{idir}/gauge/G{i+1:02d}_observed_q.csv")['q_day'].values
    ## Plot the observed discharge
    if i+1 in [2, 9, 13, 18]: # Plot only selected gauges
        ax = fig.add_subplot(2,2,i_vis+1)
        ax.plot(range(len(q_obs)), q_obs, linewidth=1, color='black', label='Observed', zorder=2)
        ax.text(0.31, 1.085, "CC: ", ha='center', color='black', transform=ax.transAxes, size=28)
        ax.text(0.57, 1.085, "rBIAS: ", ha='center', color='black', transform=ax.transAxes, size=28)
        ax.text(0.84, 1.085, "NSE: ", ha='center', color='black', transform=ax.transAxes, size=28)
        ax.set_title(f"G{i+1}", fontsize=32, loc='left')
        ax.tick_params(labelsize=28)
        i_vis += 1
        if i+1 == 2:
            ax.set_ylabel("Discharge [$m^3/s$]", size=32, math_fontfamily='cm')
        if i+1 == 13:
            ax.set_xlabel("Time", size=32, math_fontfamily='cm')
    ## Plot the predicted discharge
    colors = ["blue", "red"]
    text_heights = [1.085, 1.015]
    for j, (color, exp, text_height) in enumerate(zip(colors, exps, text_heights)):
        ## Load the BAM predicted discharge
        q_pred = pd.read_csv(f"{idir}/gauge/G{i+1:02d}_predicted_q_{exp}.csv")['q_day'].values
        ## Calculate the skill scores
        cc = stats.pearsonr(q_obs, q_pred)[0]
        rbias = rbias_func(q_obs, q_pred)
        nse = nse_func(q_obs, q_pred)
        ## Save the skill scores
        df.loc[f'CC_{exp}', f'G{i+1}'] = cc
        df.loc[f'rBIAS_{exp}', f'G{i+1}'] = rbias
        df.loc[f'NSE_{exp}', f'G{i+1}'] = nse
        ## Plot
        if i+1 in [2, 9, 13, 18]: # Plot only selected gauges
            if exp == "conventional":
                ax.plot(range(len(q_obs)), q_pred, linewidth=1.5, linestyle='--', color=color, label="Original", zorder=3)
            else:
                ax.plot(range(len(q_obs)), q_pred, linewidth=1.5, linestyle='--', color=color, label="Irrg.-corrected", zorder=3)
            ax.text(0.41, text_height, f"{cc:.2f}", ha='center', color=color, transform=ax.transAxes, size=28)
            ax.text(0.70, text_height, f"{rbias:.2f}", ha='center', color=color, transform=ax.transAxes, size=28)
            ax.text(0.95, text_height, f"{nse:.2f}", ha='center', color=color, transform=ax.transAxes, size=28)
            ## Acquire the maximum discharge for ylim setting
            if j == 0:
                q_max = np.max([q_obs.max() , q_pred.max()])
            ## ylim
            ax.set_ylim([0, q_max*1.05])
            if i+1 == 18:
                ax.legend(loc="upper right", fontsize=28)

## Save figure
# if not os.path.isdir(odir):
#     os.makedirs(odir)
# fig.savefig(f"{odir}/Fig2.png")       

In [ ]:
# ======================================================================
# Figure 3b: Boxplots of three skill metrics
# ======================================================================
## Settings
metrics = ["CC", "rBIAS", "NSE"]
vars = ["All", "Up", "Down"]
exp_colors = ['lightblue', 'pink']

## Separete the evaluation dataframe into upstream and downstream
df_up = df.iloc[:, 0:10]    # G1 - G10
df_down = df.iloc[:, 10:]   # G11 - G19

## Iterative visualization for each skill metric
fig = plt.figure(figsize=(18,8), facecolor='w', dpi=100, tight_layout=True)
for i, metric in enumerate(metrics):
    ## Summarize the evaluation scores
    eval_scores = list()
    eval_scores.append(df.loc[f'{metric}_conventional'].to_list())
    eval_scores.append(df.loc[f'{metric}_virtual'].to_list())
    eval_scores.append(df_up.loc[f'{metric}_conventional'].to_list())
    eval_scores.append(df_up.loc[f'{metric}_virtual'].to_list())
    eval_scores.append(df_down.loc[f'{metric}_conventional'].to_list())
    eval_scores.append(df_down.loc[f'{metric}_virtual'].to_list())
    ## CC
    ax = fig.add_subplot(1, 3, i+1)
    bplot1 = ax.boxplot(eval_scores,
                showmeans=True,
                widths=0.65,
                positions=[0,1,2.5,3.5,5,6],
                medianprops=dict(color="black", linewidth=3),
                meanprops=dict(marker="x", markeredgecolor="red", markersize=12),
                patch_artist=True
    )
    for j, patch in enumerate(bplot1['boxes']):
        patch.set_facecolor(exp_colors[j%2])
    ax.grid(axis="y", c="gainsboro")
    ax.set_xticks(np.arange(0.5, 6, 2.5, dtype=float))
    ax.set_xticklabels(vars)
    ax.set_ylabel(f"{metric}", fontdict=dict(size=32))
    ax.tick_params(labelsize=28, bottom=False)
    ## Calculate mean improvements in each scores
    delta_score = df.loc[f'{metric}_virtual'].mean() - df.loc[f'{metric}_conventional'].mean()
    print(f"Mean improvements in {metric} (ALL): ", delta_score)
    delta_score_up = df_up.loc[f'{metric}_virtual'].mean() - df_up.loc[f'{metric}_conventional'].mean()
    print(f"Mean improvements in {metric} (UP): ", delta_score_up)
    delta_score_down = df_down.loc[f'{metric}_virtual'].mean() - df_down.loc[f'{metric}_conventional'].mean()
    print(f"Mean improvements in {metric} (DOWN): ", delta_score_down)

In [ ]:
# ======================================================================
# Figure 4
# ======================================================================
for exp in exps:
    ## Load the results
    df = pd.read_csv(f"{idir}/reach_results_{exp}.csv")
    df = df[df['SectionID'] != 0].reset_index(drop=True)
    ## Colormap setting
    colors = [(0, "purple"), (0.25, 'dodgerblue'), (0.5, "white"), (0.75, "red"), (1.0, "darkred")]
    cmap = LinearSegmentedColormap.from_list("custom_cmap", colors)
    ## Draw the scatter plot (width CV vs Posterior uncertainty)
    fig = plt.figure(figsize=(10,8), facecolor='w', dpi=100, tight_layout=True)
    ax = fig.add_subplot(111, fc='gray')
    im = ax.scatter(df['WidLgCV'], df['PostRelUnc'], marker="o", s=20, c=np.abs(df['DisRelGap'])*100, cmap=cmap, vmin=0, vmax=200, alpha=0.7, zorder=3)
    ax.text(0.60, 0.93, f"Mean RE: {df['DisRelGap'].mean():.3f}", ha='left', color='white', transform=ax.transAxes, size=24)
    ax.text(0.60, 0.87, f"Median RE: {df['DisRelGap'].median():.3f}", ha='left', color='white', transform=ax.transAxes, size=24)
    ## Acceptable ranges
    if exp == "conventional":
        widlgcv_limit = 0.095
        postrelunc_limit = 0.485
    else:
        widlgcv_limit = 0.090
        postrelunc_limit = 0.845
    ax.plot([widlgcv_limit,widlgcv_limit], [0.000,2.000], linewidth=1.5, linestyle='--', color="lightyellow", zorder=2)
    ax.plot([0.000,0.225], [postrelunc_limit,postrelunc_limit], linewidth=1.5, linestyle='--', color="lightyellow", zorder=2)
    ax.fill_between([widlgcv_limit,0.225], [0.000,0.000], [postrelunc_limit,postrelunc_limit], fc="darkgray", zorder=1)
    ax.set_xlim(left=0.000, right=0.225)
    ax.set_ylim(bottom=0, top=2)
    ax.set_xlabel("log(w) CV", size=32, math_fontfamily='cm')
    ax.set_ylabel("Posterior uncertainty", size=32, math_fontfamily='cm')
    ax.tick_params(labelsize=28)
    cbar = plt.colorbar(im, extend='max', pad=0.025)
    cbar.set_label("Relative error [%]", fontsize=32)
    cbar.ax.tick_params(labelsize=28)
    ## Save figure
    fig.savefig(f"{odir}/Fig4_{exp}.png")